let us start by importing the required functions

In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
# import statsemodels.api as sm
import SALib
import seaborn as sns
import mpl_toolkits.axisartist as AA
import mpld3
import scipy as sp
import copy
import matplotlib.ticker as ticker

from ema_workbench import (Model, RealParameter, IntegerParameter, Policy, CategoricalParameter, Constant, TimeSeriesOutcome, ScalarOutcome, perform_experiments, ema_logging, save_results, load_results)
from ema_workbench.em_framework.evaluators import LHS, SOBOL, MORRIS, SequentialEvaluator
from ema_workbench.connectors.pysd_connector import PysdModel 
from ema_workbench.connectors import vensimDLLwrapper
from ema_workbench.connectors.vensim import VensimModel
from ema_workbench.em_framework.samplers import sample_levers, sample_uncertainties
from ema_workbench.em_framework.salib_samplers import get_SALib_problem
from ema_workbench.util import load_results
from ema_workbench.util import ema_logging

from ema_workbench.analysis import prim
from ema_workbench.analysis.plotting import lines, multiple_densities, kde_over_time
from ema_workbench.analysis.plotting_util import KDE
from ema_workbench.analysis.plotting_util import determine_time_dimension
from ema_workbench.analysis.pairs_plotting import pairs_scatter

from SALib.sample import saltelli
from SALib.analyze import sobol, morris
from SALib.test_functions import Ishigami

from mpl_toolkits.axes_grid1 import host_subplot

from scipy.stats import linregress

ema_logging.log_to_stderr(ema_logging.INFO)

C:\Users\guri_\Anaconda3\lib\site-packages\ema_workbench\connectors\__init__.py:18: ImportWarning: netlogo connector not available
  warnings.warn("netlogo connector not available", ImportWarning)


<Logger EMA (DEBUG)>

# Adding Reconversion Open Exploration  

In [2]:
vensimModel = VensimModel("Step4Modellong", model_file="ShortTermPolicyforObjectives_Step4_Full Policies.vpm" )

In [3]:
vensimModel.uncertainties = [

            RealParameter("Average Fertility Rate", 0.01, 0.025),
            RealParameter("Average Mortality Rate", 0.00126, 0.00154),
            IntegerParameter("BASE Average Household size", 2, 7),     
            RealParameter("average Immigration percent", 0.01, 0.06),
            RealParameter("Housing Area Per person", 0.000004, 0.000006),
            RealParameter("Floor Area Ratio", 2, 7),
            RealParameter("Average Residential approval period", 0.5, 2),
            RealParameter("Base Green Area commisioned per year", 0.02, 0.06),
            RealParameter("average green area construction period", 0.5, 2),
            RealParameter("Base Economic area Conversion rate", 0.01, 0.06),
            RealParameter("Base Slum reconversion rate", 0.01, 0.06),
            RealParameter("Fraction of people selling houses when leaving city", 0.1, 0.5),
            RealParameter("house construction time", 0.5, 2),
            IntegerParameter("time delay effect uncertainty of spending capacity on housing price", 1, 4),
            RealParameter("Slum dwelling area per person", 0.000004, 0.000006),
            RealParameter("Base residential conversion rate", 0.01, 0.05),
            RealParameter("average Emmigration percent", 0.008, 0.05),
            RealParameter("Relocation Rate", 0.00, 0.02), #for relocation of population on streets
            RealParameter("Average Commercial Approval Period", 0.5, 2),
            RealParameter("Base Green Area Reconversion rate", 0.001, 0.03),
            RealParameter("Average Slum Construction period", 0.1, 0.5),
            RealParameter("Average slum Demolition Period", 0.5, 2),
            RealParameter("uncertainty of average percent of inflation effect", 0.01, 0.08),
            RealParameter("average payment to income ratio for housing", 0.2, 0.5),
            RealParameter("Base Non Economic Reconv rate", 0.01 ,0.05),
            RealParameter("Ratio of Renting to Buying",0.2, 1),
            RealParameter("margin of empty housing",0.1,0.5)
]

vensimModel.levers= [
            RealParameter("Policy High rise Program switch",0,1),
            RealParameter("increase in FAR",1,5),
            RealParameter("FAR implementation year",2020,2050),
    
            RealParameter("Accelerated city growth program switch",0,1),
            RealParameter("effect of green recov",1,5),
            RealParameter("Green reconv policy start",2020,2050),
    
            RealParameter("Policy reconversion of Slums Switch",0,1),
            RealParameter("effect of reconversion rate on Policy",1,5),
            RealParameter("Slum reconv polict start year",2020,2050),
    
            RealParameter("Austerity Policy Program Switch",0,1),
            RealParameter("effect on non resi reconv of policy",1,5),
            RealParameter("reconv policy start year",2020,2050),
            RealParameter("Austerity effect on inflation",1,5),
            RealParameter("Auterity implementation year",2020,2050),
    
            RealParameter("Housing Finance Policy Program Switch",0,1),
            RealParameter("Housing finance effect on household spendings",1,5),
            RealParameter("Housing Finane Policy start year",2020,2050),
    
            RealParameter("Policy Relocation program Switch",0,1),
            RealParameter("effect on relocation rate",1,5),
            RealParameter("additional relocation housing factor",1,5),
            RealParameter("Relocation policy start year",2020,2050),
]

vensimModel.outcomes = [
    TimeSeriesOutcome('Housing Affordability Rating'),
    TimeSeriesOutcome('People with No Settlements'),
    TimeSeriesOutcome('Illegal Slum Population'),
    TimeSeriesOutcome('Urban Non Residential Area'),
    TimeSeriesOutcome('Green Area')
    
]
    
MAXIMIZE = ScalarOutcome.MAXIMIZE   
MINIMIZE = ScalarOutcome.MINIMIZE 

vensimModel.robustness_minimax_maximin = [ScalarOutcome('Housing Affordability Rating' , kind=MINIMIZE, function=np.max),
                                          ScalarOutcome('People with No Settlements', kind=MINIMIZE, function=np.max),
                                          ScalarOutcome('Illegal Slum Population', kind=MINIMIZE, function=np.max),
                                          ScalarOutcome('Urban Non Residential Area', kind=MAXIMIZE, function=np.min),
                                          ScalarOutcome('Green Area', kind=MAXIMIZE, function=np.min)
                                          ]

In [4]:
n_scenarios = 50
scenarios = sample_uncertainties(vensimModel, n_scenarios)
nfe = 1000
    
with SequentialEvaluator(vensimModel) as evaluator:
    b = evaluator.robust_optimize(vensimModel.robustness_minimax_maximin, scenarios, nfe=nfe, population_size=25,
                                  epsilons=[0.1,]*len(vensimModel.robustness_minimax_maximin) )

[MainProcess/INFO] performing 50 scenarios * 25 policies * 1 model(s) = 1250 experiments
[MainProcess/INFO] performing experiments sequentially
[MainProcess/INFO] 417 cases completed
[MainProcess/INFO] 834 cases completed
[MainProcess/INFO] experiments finished
[MainProcess/INFO] performing 50 scenarios * 26 policies * 1 model(s) = 1300 experiments
[MainProcess/INFO] performing experiments sequentially
[MainProcess/INFO] 433 cases completed
[MainProcess/INFO] 866 cases completed
[MainProcess/INFO] 1299 cases completed
[MainProcess/INFO] experiments finished
[MainProcess/INFO] performing 50 scenarios * 26 policies * 1 model(s) = 1300 experiments
[MainProcess/INFO] performing experiments sequentially
[MainProcess/INFO] 433 cases completed
[MainProcess/INFO] 866 cases completed
[MainProcess/INFO] 1299 cases completed
[MainProcess/INFO] experiments finished
[MainProcess/INFO] performing 50 scenarios * 26 policies * 1 model(s) = 1300 experiments
[MainProcess/INFO] performing experiments seq

In [5]:
b.to_csv('./Results_24_solutions_minimax_maximin_Longpolicies4.csv')

In [6]:
df_minimax_maximin = pd.DataFrame.from_csv('./Results_24_solutions_minimax_maximin_Longpolicies4.csv')

C:\Users\guri_\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  if __name__ == '__main__':


In [7]:
df_minimax_maximin.head()

,Policy High rise Program switch,increase in FAR,FAR implementation year,Accelerated city growth program switch,effect of green recov,Green reconv policy start,Policy reconversion of Slums Switch,effect of reconversion rate on Policy,Slum reconv polict start year,Austerity Policy Program Switch,...,Housing finance effect on household spendings,Housing Finane Policy start year,Policy Relocation program Switch,effect on relocation rate,additional relocation housing factor,Housing Affordability Rating,People with No Settlements,Illegal Slum Population,Urban Non Residential Area,Green Area
0,0.144584,1.297207,2037.142357,0.686893,4.101619,2036.119176,0.984451,4.894527,2029.420115,0.143184,...,4.262764,2025.280221,0.366924,1.493793,2.437766,15.499986,66431564.0,600000.0,30.055624,2.331358
1,0.144549,1.424039,2037.145704,0.794548,4.591607,2039.066102,0.999501,3.990831,2021.830249,0.465252,...,4.385703,2022.892300,0.991704,1.493269,3.512685,14.754694,39559080.0,600000.0,26.917036,3.216594
2,0.144549,1.424039,2037.145704,0.794548,4.082046,2040.153290,0.999501,4.932892,2025.220800,0.465252,...,4.385703,2022.825294,0.991704,1.493278,3.512685,14.746411,39714176.0,600000.0,28.267292,3.201644
3,0.193909,1.451411,2032.789478,0.805685,4.076270,2035.652543,0.991442,4.845277,2021.750855,0.500279,...,4.028813,2022.910488,0.187633,1.493323,2.488862,15.962681,43441324.0,600000.0,29.145554,2.352507
4,0.113617,2.869702,2037.140418,0.695509,4.591211,2040.150280,0.992533,4.577295,2022.068350,0.178644,...,3.782401,2022.001253,0.928255,1.546005,3.732784,16.828999,38594076.0,600000.0,27.700100,3.269621


In [8]:
import plotly

In [9]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import *

init_notebook_mode(connected=True)

In [10]:
# create the parcoord

df_parallel = df_minimax_maximin

data = [
    Parcoords(
        line = dict(color = df_parallel['Housing Affordability Rating'],
                   colorscale = 'Jet',
                   showscale = True,
                   reversescale = False,
                   cmin = df_parallel['Housing Affordability Rating'].min(),
                   cmax = df_parallel['Housing Affordability Rating'].max()),

        dimensions = list([
             dict(label ='People with No Settlements'  ,
                values =df_parallel['People with No Settlements']  ),
            dict(label ='Illegal Slum Population'  ,
                values =df_parallel['Illegal Slum Population']  ),
            dict(label = 'Urban Non Residential Area',
                values = df_parallel['Urban Non Residential Area']),
            dict(label = 'Housing Affordability Rating',
                values = df_parallel['Housing Affordability Rating']),
            dict(label = 'Green Area',
                values = df_parallel['Green Area'])
            
        ])
    )
]


layout = Layout(
    title = "Parallel coordinates plot"

)
fig = Figure(data=data, layout=layout)
iplot(fig)